In [ ]:
import pandas as pd
import numpy as np
from multiprocessing import Pool, Value
import time
import datetime
from math import log10

df = pd.read_json('allthroughputs.json')

def calculate_windows(link):
    start=datetime.datetime(2016,03,01,0)
    # 1464 hours = all the hours between 03/01 00:00 and 04/31 23:00
    #print link
    maxthr = {}
    maxthr[link] = {}
    #print 'Processing link %s' % link
    s = time.time()
    # 840 hours = 35 days
    # only pair window sizes
    for win in range(4,841,2):
        for delta in pd.bdate_range(start=start, periods=1464, freq='H'):
            try:
                m = np.max(df[link][delta:delta+win].values)
            except ValueError:
                print delta, delta+win
            src,dst = link.split(':')
            try:
                c = 6 - int(log10(m))
            except ValueError:
                c = 10
            if src == dst:
                c = 0
            try:
                maxthr[link][delta][str(win)] = (m, min(max(0,c),10))
            except KeyError:
                maxthr[link][delta] = {}
                maxthr[link][delta][str(win)] = (m, min(max(0,c),10))
    #print 'Link %s done in %.02f min. Writing file' % (link, (time.time() - s)/60.)
    s1 = time.time()
    targetdf = pd.DataFrame(maxthr)
    targetdf.to_json('windata/closeness--%s__%s--.json'%(src, dst))
    print 'Writen in %.02f. Link %s done in %.02f min.' % ((time.time() - s1)/60., link, (time.time() - s)/60.)

if __name__ == '__main__':
    p = Pool(24)
    s = time.time()
    l = p.map(calculate_windows, list(df.keys()))
    print 'Done in %.02f min. Writing file.' % ((time.time() - s)/60.)